In [1]:
# extracts an inventory table from Schroeder metadata file 

In [2]:
import os,glob,sys
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

lines =  open('/users/staff/leo/tables/vapor.library.1', 'r').readlines()

wmos, lats, lons, stations, dates = [],[],[],[],[]
dates_min, dates_max = [],[] 

last_wmo = ''

# appending only valid latitude and longitude data
valid_data_indices = []

for l,ind in zip( lines, range(len(lines)) ):
    #print (l)
    s = l.split(':')
    
    wmo = s[0]
    if wmo != last_wmo:

        lat = s[1]
        lon = s[2]
        d = s[5]
        stat = s[7]

        try:
            lats.append(float(lat))
            lons.append(float(lon))
            valid_data_indices.append(ind)
             
        except:
            lats.append(lat)
            lons.append(lon)
             
        wmos.append(wmo)

        stations.append(stat.replace('\n', ''))

    
        y = d[0:4]
        m = d[4:6]
        da = d[6:8]
        
        if m == '00':
            m = '01'
        if da == '00':
            da ='01'
        try:
            date = pd.to_datetime(y+m+da)
        except:
            date = d[0:8]

        dates.append(date)

    #last_wmo = wmo
    

        
#dic = {'wmo':wmos , 'lat':lats, 'lon':lons, 'dates_min': dates_min, 'dates_min': dates_max }
dic = {'wmo':wmos , 'lat':lats, 'lon':lons, 'stations':stations, 'dates': dates }

df_all = pd.DataFrame.from_dict( dic )
             
df_clean = df_all.iloc[valid_data_indices]

/tmp/ipykernel_2317641/3480537501.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:
len(df_clean)

40458

In [4]:
df_clean = df_clean.loc[(df_clean.lat < 90) & (df_clean.lat > -90) & (df_clean.lon < 180) & (df_clean.lat > -180) ]

In [5]:
df_clean.head(20) 

,wmo,lat,lon,stations,dates
0,01001,70.98,-8.35,"Jan Mayen - Oldest Meteorological Station (Eldstemetten), Norway",1922-01-01 00:00:00
1,01001,71.012,-8.462,"Jan Mayen - ""Atlantic City"", Norway",1948-01-01 00:00:00
2,01001,71.0142,-8.4602,"Jan Mayen - Old Meteorological Station (Gamlemetten) and Frydenlund House, Norway",1949-01-01 00:00:00
3,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1962-01-01 00:00:00
4,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",0
5,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1975-10-01 00:00:00
6,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1976-11-24 00:00:00
7,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1983-02-01 00:00:00
8,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1983-04-15 00:00:00
9,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1994-08-07 00:00:00


In [6]:
import numpy as np
uniques = np.unique(df_clean.wmo)
from tqdm import tqdm

# rerun to reduce to single unique stations wmo ids
wmos, lats, lons, stations, dates_min, dates_max = [],[],[],[],[], []

for s in tqdm(uniques):
    d = df_clean[df_clean.wmo == s]
    try:
        min_d, max_d = d.dates.values[0], d.dates.values[-1]
        wmos.append(s)
        stations.append(d.stations.values[0])
        lats.append(d.lat.values[0])
        lons.append(d.lon.values[0])
    except:
        print(d)
    try:
        dates_min.append( min([l for l in d.dates.values if isinstance(l, pd.Timestamp)] ))
        dates_max.append( max([l for l in d.dates.values if isinstance(l, pd.Timestamp)] ))
    except:
        dates_max.append('9999999')
        dates_min.append('9999999')

100%|██████████| 4088/4088 [00:08<00:00, 462.68it/s]


In [7]:
dic = {'wmo':wmos , 'lat':lats, 'lon':lons, 'station':stations, 'date_min': dates_min ,  'date_max': dates_max }
sch_df = pd.DataFrame.from_dict( dic )
sch_df.head(10)

sch_df.to_csv('schroeder_metadata.csv', sep = '\t')

In [8]:
sch_df[sch_df.wmo == '01733']

,wmo,lat,lon,station,date_min,date_max


In [9]:
### create WMO table 
import glob
import regex as re

wmo_tabs = glob.glob('/users/staff/leo/tables/world-weather-records/*WWR*.txt')



In [10]:
!conda install -c conda-forge regex

/bin/bash: conda: command not found


In [11]:
wmo_tabs

['/users/staff/leo/tables/world-weather-records/WWR_v01_global_array_1961-1970.txt',
 '/users/staff/leo/tables/world-weather-records/WWR_v01_global_array_1971-1980.txt',
 '/users/staff/leo/tables/world-weather-records/WWR_v01_global_array_1981-1990.txt',
 '/users/staff/leo/tables/world-weather-records/WWR_v02_global_StnLst.txt',
 '/users/staff/leo/tables/world-weather-records/StnLstWWR_Region00_2001-2010.txt',
 '/users/staff/leo/tables/world-weather-records/StnLstWWR_Region01_2001-2010.txt',
 '/users/staff/leo/tables/world-weather-records/StnLstWWR_Region02_2001-2010.txt',
 '/users/staff/leo/tables/world-weather-records/StnLstWWR_Region03_2001-2010.txt',
 '/users/staff/leo/tables/world-weather-records/StnLstWWR_Region04_2001-2010.txt',
 '/users/staff/leo/tables/world-weather-records/StnLstWWR_Region05_2001-2010.txt',
 '/users/staff/leo/tables/world-weather-records/StnLstWWR_Region06_2001-2010.txt',
 '/users/staff/leo/tables/world-weather-records/StnLstWWR_Region06_2011-2016.txt',
 '/us

In [68]:
import copy

""" Copied from Leo script """

tabns = glob.glob('/users/staff/leo/tables/world-weather-records/*WWR*.txt')
dlist = {}
for t in tabns:
    with open(t,encoding='ISO-8859-1') as f:
        try:
            data = f.read().split('\n')
            gdata = []
            for d in data:
                if re.search('[a-zA-Z]',d):
                    gdata.append(d)
            dlist[t.split('/')[-1].split('.')[0]] = copy.copy(gdata)
        except Exception as e:
            print(t.split('/')[-1].split('.')[0], e)

stns = {'id': [],'name': [],'lats': [], 'lons': [] }
latsign = {'N':1, 'S':-1}
lonsign = {'E':1, 'W':-1}
for d,v in dlist.items():
    for vv in v:
        if vv[2:7] not in stns['id']:
            
            try:
                
                if vv[7] == ' ':
                    h = vv[10:21]
                    h = '0'.join(h.split(' '))
                    vv=vv[:10]+h+vv[21:]

                    name = vv[48:73]
                    lats = (int(vv[10:12]) + int(vv[12:14]) / 60) * latsign[vv[14]]
                    lons = (int(vv[16:19]) + int(vv[19:21]) / 60) * lonsign[vv[21]]

                        
                else:
                    h = vv[8:18]
                    h = '0'.join(h.split(' '))
                    vv=vv[:8]+h+vv[18:]

                    name = vv[43:68]
                    lats = (int(vv[8:10]) + int(vv[10:12]) / 60) * latsign[vv[12]]
                    lons = (int(vv[13:16]) + int(vv[16:18]) / 60) * lonsign[vv[18]]


            except:
                print('failed', vv)
                continue
            stns['id'].append(str(vv[2:7]))
            stns['name'].append(name)
            stns['lats'].append(lats)
            stns['lons'].append(lons)


failed   6003012756N01523NCANARY ISLANDS          LAS PALMAS/GANDO           24        031036405
failed   9685310747511026EINDONESIA               JOGYAKARTA                107        804626115
failed   9619510138510339EINDONESIA               JAMBI                      25        805204605
failed   Wmo#  # 0Lat000Lon0  Country Name             Station Name               (m)  (.1m)    #       :   Pres Pres Temp Prec Temp Temp   RH| Obs 
failed   96A54 1 00000000000  INDONESIA                INDRAPURI                                        :      0    0   30   31   30   29    0  120
failed   96B63 1 00000000000  INDONESIA                TUNTUNGAN MEDAN                                  :      0    0   18   17   18   17    0   70
failed   96C01 1 00000000000  INDONESIA                PADANG PANJANG                                   :      0    0   12   11   12   12    0   47
failed   96D05 1 00000000000  INDONESIA                LUBUK RUSO                                       :      0    

In [69]:
wmo = pd.DataFrame.from_dict( stns )



In [70]:
wmo.head(5)

,id,name,lats,lons
0,43333,PORT BLAIR,11.666667,92.716667
1,96996,COCOS ISLAND,-12.200000,96.833333
2,67005,DZAOUDZI/PAMANZI,-12.800000,45.283333
3,67001,MORONI,-11.700000,43.233333
4,61968,ILES GLORIEUSES,-11.550000,47.283333


In [24]:
wmo.loc[wmo.id == '11035']

,id,name,lats,lons,elevation
817,11035,WIEN-HOHE WARTE 0,48.25,16.366667,001990


In [95]:
# getting elevation
f = '../data/world-weather-records/WWR_v02_global_StnLst.txt'
lines = open(f, 'r').readlines()
d = {'id':[], 'elevation':[] }
for l in lines[5:]:
    id = l[2:7].strip('')
    elev = l[74:78]
    elev_b = l[79:83]
    
    try:
        id = int(id)
    except:
        continue
    
    d['id'].append(str(id))
    
    if elev_b:
        d['elevation'].append(elev_b)
    else:
        d['elevation'].append(elev)
        
df = pd.DataFrame.from_dict(d)



In [108]:
elev = []
for i in wmo.id:
    dr = df.loc[df.id == i]
    if len(dr) >0:
        p = dr.elevation.values[0]
    else:
        p = -999
    if p:
        try:
            elev.append(float(p))
        except:
            elev.append(-999)
        

In [109]:
elev[:5]

[78.0, 4.0, 7.0, -999.0, -999.0]

In [110]:
wmo['elevation'] = elev

In [111]:
wmo.head(5)

,id,name,lats,lons,elevation
0,43333,PORT BLAIR,11.666667,92.716667,78.0
1,96996,COCOS ISLAND,-12.200000,96.833333,4.0
2,67005,DZAOUDZI/PAMANZI,-12.800000,45.283333,7.0
3,67001,MORONI,-11.700000,43.233333,-999.0
4,61968,ILES GLORIEUSES,-11.550000,47.283333,-999.0


In [112]:
wmo.to_csv('wmo_metadata.csv', sep = '\t')